In [ ]:
from matplotlib import pyplot as plt

In [ ]:
from src.features.build_features import load_raw

df_all = load_raw()
df_all.keys()

## plots of target

In [ ]:
j = 'labels_train'
k = 'total_cases'
(df_all[j][k]+1).groupby('city').plot(figsize=(20,3), logy=True)
plt.legend()
plt.title("%s/%s"%(j,k))
plt.show()

df_all[j][k].groupby('city').plot(figsize=(20,3), logy=False)
plt.legend()
plt.title("%s/%s"%(j,k))
plt.show()

## construct half sin

In [ ]:
# https://stackoverflow.com/a/22576622/4126114
import numpy as np
import pandas as pd

# 5 peaks within 8000 points
# Fs, f, sample, dlt = 8000, 5, 8000, 0

Fs, f, sample, dlt = 26, 1, 26, -1*np.pi/2

x = np.arange(sample)
y = np.sin(2 * np.pi * f * x / Fs + dlt)
z = pd.Series(y, index=x)

# z.head()

# plt.plot(x, y)
z.plot()
plt.xlabel('sample(n)')
plt.ylabel('voltage(V)')
plt.show()

## correlate total cases with half-sin

In [ ]:
df_all['labels_train']['log_cases'] = np.log(df_all['labels_train']['total_cases']+1)

df_all['labels_train']['corr_signature'] = np.nan
for city in ['sj','iq']:
    df_all['labels_train'].loc[city,'corr_signature'] = np.correlate(
        df_all['labels_train'].loc[city, 'log_cases'],
        z,
        "same"
    )/3 + 1 # scaling for visualization
    
threshold = 2 # depends on scaling
df_all['labels_train']['is_epidemic'] = df_all['labels_train']['corr_signature'].apply(lambda row: row > threshold)

df_all['labels_train'].head(n=2)

In [ ]:
for city in ['sj','iq']:
    df_all['labels_train'].loc[city,['log_cases', 'corr_signature']].plot(figsize=(20,3))
    plt.title(city)
    plt.show()
    
    df_all['labels_train'].loc[city,['total_cases']].plot(style='.-', figsize=(20,3))
    (
        df_all['labels_train'].loc[city,['total_cases', 'is_epidemic']].apply(
            lambda row: row['total_cases'] if row['is_epidemic'] else np.nan,
            axis=1
        )
    ).plot(style='.', figsize=(20,3))
    plt.title(city)
    plt.show()

## check that `is_epidemic` is smooth

In [ ]:
for city in ['sj','iq']:
    df_all['labels_train'].loc[city, 'is_epidemic'].astype('int').plot(figsize=(20,3))
    plt.legend()
    plt.title(city)
    plt.show()

## save to file

In [ ]:
import os
fn = 'data/processed/is_epidemic.pkl'
if os.path.exists(fn):
    raise ValueError("File exists: %s"%fn)
    
df_all['labels_train'].to_pickle(fn)